In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import correlate

In [2]:
data = np.load('eNodeB_a.npy')

In [3]:
minimum = 500
data = np.array([user for user in data if user.shape[0] > minimum])
mcs = []

for user in data:
    mcs.append(user[:,4])
    
mcs = np.array(mcs)

minimum = 0
maximum = 31

#mcs = list((mcs-minimum)/(maximum-minimum))
for i in range(len(mcs)):
    mcs[i] = mcs[i]-mcs[i].mean()

In [ ]:
def compute_distance(s1, s2, norm='l2'):
    
    distance = None
    
    if norm == 'l1':
        # MAE criterion
        distance = np.mean(np.abs(s1-s2))
    elif norm == 'l2':
        # MSE criterion
        distance = np.mean(np.power((s1-s2),2))
    
    return distance


dist = np.zeros((len(mcs),len(mcs)))

for i in tqdm(range(len(mcs)-1)):
    for j in range(i+1,len(mcs)):
        s1 = mcs[i]
        s2 = mcs[j]
        ccorr = correlate(s1,s2,'valid')
        index = ccorr.argmax()
        if len(s1) > len(s2):
            s1 = s1[index:index+len(s2)]
        else:
            s2 = s2[index:index+len(s1)]    
        dist[i,j] = compute_distance(s1,s2,'l1')
        
i_lower = np.tril_indices(dist.shape[0], -1)
dist[i_lower] = dist.T[i_lower]

  1%|▉                                                                               | 17/1463 [00:07<08:57,  2.69it/s]

In [ ]:
plt.imshow(dist)

In [ ]:
dist_n = (dist-dist.min())/(dist.max()-dist.min())
binary = dist_n < 0.35
plt.imshow(binary)
# se vuoi dare un'occhiata anche a questo...
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import asyn_fluidc
G = nx.from_numpy_array(binary)
giant = max(nx.connected_component_subgraphs(G), key=len)
communities1 = greedy_modularity_communities(giant)
communities2 = asyn_fluidc(giant,4)
for elem in communities1:
    print(len(elem))
for elem in communities2:
    print(len(elem))

In [ ]:
from sklearn.cluster import SpectralClustering

# creation of similarity matrix
# stai attento a questo parametro. se cala viene un unico cluster, se aumenta troppo anche
delta = 7
adj = np.exp(- dist ** 2 / (2. * delta ** 2))
# clustering
spectral = SpectralClustering(3,affinity='precomputed')
labels = spectral.fit_predict(adj)
plt.imshow(adj)

In [ ]:
c1 = data[labels==0]
c2 = data[labels==1]
c3 = data[labels==2]


a = np.random.randint(len(c1))
b = np.random.randint(len(c2))
c = np.random.randint(len(c3))


plt.figure()
plt.plot(c1[a][:,4])

plt.figure()
plt.plot(c2[b][:,4])

plt.figure()
plt.plot(c3[c][:,4])

print(len(c1), len(c2), len(c3))